<!--NAVIGATION-->
< [Introduction](Introduction.ipynb) | [Contents](Contents.ipynb) | [Condition 1](Condition_1.ipynb) >

# 2. Preparation
## 2.1 Introduction
For this task you are provided with the following raster layers: `buildg.tif`, `roads.tif`, `dtm.tif`, and `gwlevel.tif`. You can find these files in the `data` folder.

To work with PCRaster we need to convert the files to the PCRaster format. You can do that from the command line using the GDAL commands. Here we'll do the conversion with the GDAL Python library.

## 2.2 Convert rasters with GDAL

Let's convert the `buildg.tif` file.
First we import `gdal` and `gdalconst` from the `osgeo` library:

In [ ]:
from osgeo import gdal, gdalconst

Then we read the GeoTIFF from our `data` folder.

In [ ]:
src_ds = gdal.Open( "data/buildg.tif" )

Next we can use the Python equivalent of [`gdal_translate`](https://gdal.org/programs/gdal_translate.html#gdal-translate): `Translate`. We need to use the name of the format. See [GDAL Raster drivers (short name)](https://gdal.org/drivers/raster/index.html) for formats. In our case the output format is `PCRaster`.

We also need to define the data type of the output.

PCRaster uses data typing of the data in the database: each map has one of the six data types used attached to it. These data types help you and PCRaster to structure the data. See the table below.<br>

| data type | description of attributes | domain | example |
|:-------:|:-------:|:-------:|:-------:|
| boolean | boolean | 0 (FALSE),<br> 1 (TRUE) | suitable/unsuitable, visible/non visible |
| nominal |	classified, no order | whole values | soil classes |
| ordinal | classified, order | whole values | succession stages, income groups |
| scalar | continuous, linear | real values | temperature, concentration |
| directional | continuous, directional | 0 to 360 degrees | aspect |
| ldd | direction to neighbour cell | codes of directions | drainage networks |

In the arguments of `Translate` you need to define the `outputType` and `metadataOptions` based on the table below. Note that we can only convert boolean, nominal and scalar rasters. The ldd and directional rasters are created within PCRaster. For scalar we can use Float 32 or Float 64, depending on the desired precision.

| data type | `outputType` | `metadataOptions` |
|:-------:|:-------:|:-------:|
| boolean | `gdalconst.GDT_Byte` | `'VS_BOOLEAN'`|
| nominal | `gdalconst.GDT_Int32` | `'VS_NOMINAL'`|
| scalar | `gdalconst.GDT_Float32` | `'VS_SCALAR'`|
| scalar | `gdalconst.GDT_Float64` | `'VS_SCALAR'`|

`buildg.tif` is a nominal raster. It contains classes. Therefore the code we'll use for `Translate` is:

In [ ]:
dst_ds = gdal.Translate('data/buildg.map', src_ds, format='PCRaster', \
                        outputType=gdalconst.GDT_Int32, metadataOptions='VS_NOMINAL')

And we properly close the datasets to flush to disk:

In [ ]:
dst_ds = None
src_ds = None

Now you can find the file `buildg.map` on your hard disk. Check if that's the case if you're running this notebook locally.

Let's apply the same code to the other files `dtm.tif`, `roads.tif` and `gwlevel.tif`. In this case it's easier to write it as a function.

In [ ]:
#Import gdal
from osgeo import gdal

def ConvertToPCRaster(src_filename,dst_filename,ot,VS):
    #Open existing dataset
    src_ds = gdal.Open(src_filename)
    
    #GDAL Translate
    dst_ds = gdal.Translate(dst_filename, src_ds, format='PCRaster', outputType=ot, metadataOptions=VS)
    
    #Properly close the datasets to flush to disk
    dst_ds = None
    src_ds = None
    
ConvertToPCRaster("data/buildg.tif","data/buildg.map",gdalconst.GDT_Int32,"VS_NOMINAL")
ConvertToPCRaster("data/roads.tif","data/roads.map",gdalconst.GDT_Int32,"VS_NOMINAL")
ConvertToPCRaster("data/gwlevel.tif","data/gwlevel.map",gdalconst.GDT_Float32,"VS_SCALAR")
ConvertToPCRaster("data/dtm.tif","data/dtm.map",gdalconst.GDT_Float32,"VS_SCALAR")

## 2.3 Inspecting the data
For map algebra the properties of all raster layers used in calculations need to be the same.
Let's find out if they have the same number of rows and columns, coordinates and pixel size.

Let's open a PCRaster file with GDAL.

In [ ]:
RasterLayer = gdal.Open("data/dtm.map")

To get the numbers of rows and columns we can use the GDAL functions `RasterXSize` and `RasterYSize`.
`RasterLayer.GetDescription()` gives the relative path of the layer.

In [ ]:
Path = RasterLayer.GetDescription()
NrColumns = RasterLayer.RasterXSize
NrRows = RasterLayer.RasterYSize
print('{} has {} columns and {} rows'.format(Path,NrColumns,NrRows ))

To know the coordinates and pixel size we can use `GetGeoTransform` from GDAL.
It returns a tuple with the following information:
`RasterLayer.GetGeoTransform[0]`: Top left X coordinate

`RasterLayer.GetGeoTransform[1]`: West-East pixel resolution

`RasterLayer.GetGeoTransform[2]`: Rotation, 0 for "north up"

`RasterLayer.GetGeoTransform[3]`: Top left Y coordinate

`RasterLayer.GetGeoTransform[4]`: Rotation, 0 for "north up"

`RasterLayer.GetGeoTransform[5]`: North-South pixel resolution (negative)

In [ ]:
RasterLayer.GetGeoTransform()

Let's present that more readable.

In [ ]:
print("Origin = ({}, {})".format(RasterLayer.GetGeoTransform()[0], RasterLayer.GetGeoTransform()[3]))
print("Pixel Size = ({}, {})".format(RasterLayer.GetGeoTransform()[1],RasterLayer.GetGeoTransform()[5]))

We can also use GDAL to get the projection information. The PCRaster format doesn't store the projection information, but when we converted the rasters it saves an XML file with the projection info. The GDAL function is `GetProjection()`.

In [ ]:
RasterLayer.GetProjection()

That gives us a string in the OGC WKT format.
In order to make the projection and the units readable, we need to parse the OGC WKT string. For that we'll use the [PyCRS](https://pypi.org/project/PyCRS/) library. 

If it's not installed yet if you run this notebook locally, you can install the library by typing the following at the Anaconda prompt:

`conda install -c conda-forge pycrs`

First we import the library and save the OGC WKT string in a variable that we parse.

In [ ]:
import pycrs
RasterLayerProjection = RasterLayer.GetProjection()

# Parse OGC WKT string
crs = pycrs.parse.from_ogc_wkt(RasterLayerProjection)

We can check if it is a projected coordinate system:

In [ ]:
isinstance(crs, pycrs.ProjCS)

The result is `True` so we're not dealing with a Geographical Coordinate System.
Let's get the name of the projection.

In [ ]:
ProjectionName = crs.name
print(ProjectionName)

We can also get the units of the projection:

In [ ]:
ProjectionUnits = crs.unit.unitname.ogc_wkt
print(ProjectionUnits)

Finally it would be useful to get some statistics from the raster layer. Let's calculate the minimum and maximum value.
Because raster layers can have multiple bands we need to select the band. In our case we have a single band raster layer, so we have to choose band 1.

In [ ]:
RasterLayerBand = RasterLayer.GetRasterBand(1)

Now we can calculate the minumum and maximum value of the band using respectively `GetMinimum` and `GetMaximum`.

In [ ]:
RasterLayerMinimum = RasterLayerBand.GetMinimum()
RasterLayerMaximum = RasterLayerBand.GetMaximum()
print("Minimum: ", RasterLayerMinimum)
print("Maximum: ", RasterLayerMaximum)

Let's write this in a function and get the raster layer properties for all the raster layers so we can compare.

In [ ]:
from osgeo import gdal
import pycrs

def RasterLayerProperties(RasterLayer):
    print("Raster file: {}".format(RasterLayer.GetDescription()))
    print("Driver: {}/{}".format(RasterLayer.GetDriver().ShortName,
                            RasterLayer.GetDriver().LongName))
    print("Size is {} x {} x {}".format(RasterLayer.RasterXSize,
                                    RasterLayer.RasterYSize,
                                    RasterLayer.RasterCount))
    RasterLayerProjection = RasterLayer.GetProjection()
    crs = pycrs.parse.from_ogc_wkt(RasterLayerProjection)
    print("Projection:",crs.name)
    print("Map units:",crs.unit.unitname.ogc_wkt)
    geotransform = RasterLayer.GetGeoTransform()
    if geotransform:
        print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
        print("Pixel Size = ({} {}, {} {})".format(geotransform[1],crs.unit.unitname.ogc_wkt, \
                                                   geotransform[5],crs.unit.unitname.ogc_wkt))
    RasterLayerBand = RasterLayer.GetRasterBand(1)
    print("Minimum: {}".format(RasterLayerBand.GetMinimum()))
    print("Maximum: {}".format(RasterLayerBand.GetMaximum()))
    
    print()
    RasterLayer = None
    
    
DTMLayer = gdal.Open( "data/dtm.map" )
RasterLayerProperties(DTMLayer)

BuildgLayer = gdal.Open( "data/buildg.map" )
RasterLayerProperties(BuildgLayer)

RoadsLayer = gdal.Open( "data/roads.map" )
RasterLayerProperties(RoadsLayer)

GWLevelLayer = gdal.Open( "data/gwlevel.map" )
RasterLayerProperties(GWLevelLayer)

Based on these results we can conclude that the raster layers have the same properties and are suitable for map algebra.

<!--NAVIGATION-->
< [Introduction](Introduction.ipynb) | [Contents](Contents.ipynb) | [Condition 1](Condition_1.ipynb) >